In [1]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as message
from biogeme import expressions as ex
import pandas as pd
import numpy as np
import xlsxwriter

# Calibration
## Estimation of the model parameters
quetzal_germany is being estimated using [PandasBiogeme](https://biogeme.epfl.ch/). This notebook estimates calibration parameters for the model's utility functions.
- Documentation and reference: [Bierlaire, M. (2020). A short introduction to PandasBiogeme. Technical report TRANSP-OR 200605. Transport and Mobility Laboratory, ENAC, EPFL.](https://transp-or.epfl.ch/documents/technicalReports/Bier20.pdf)
- Tutorial: https://www.youtube.com/watch?v=OiM94B8WayA

### Model formulation
The model consists of systematic utility functions, one for each mode. They comprise an alternaive-specific constant (ASC), a distance-dependent part with travel time and cost summarised as generalised cost (GC), and the accessibility

> V_i = ASC + GC(T, C, b_gc_i) + AC • b_ac_i

Index i marks the demand group. I = {'commuting' (1), 'education' (2), 'shopping/medical' (3), 'business' (4), 'private' (6)}

Note: The cost variable already includes subscriptions

In [2]:
input_path = '../input/'
model_path = '../model/'

### Prepare the database

In [3]:
df = pd.read_csv(input_path + 'transport_demand/calibration_inter-cellular_trips_MiD2017.csv')
df = df[['cost_rail', 'cost_car', 'cost_coach', 'cost_bus', 'cost_walk', 'cost_air',
         'time_rail', 'time_car', 'time_coach', 'time_bus', 'time_walk', 'time_air',
         'accessibility_rail', 'accessibility_car', 'accessibility_coach', 'accessibility_bus',
         'accessibility_walk', 'accessibility_air',
         'mode_model', 'purpose_vp', 'car_avail', 'distance']]
df.columns = ['C_RAIL', 'C_CAR', 'C_COACH', 'C_BUS', 'C_NON_MOTOR', 'C_AIR',
              'T_RAIL', 'T_CAR', 'T_COACH', 'T_BUS', 'T_NON_MOTOR', 'T_AIR',
              'AC_RAIL', 'AC_CAR', 'AC_COACH', 'AC_BUS', 'AC_NON_MOTOR', 'AC_AIR',
              'MODE', 'PURPOSE', 'CAR_AV', 'DIST']

In [4]:
inf = 1000
df = df.replace({np.inf:inf})

In [5]:
df.describe()

,C_RAIL,C_CAR,C_COACH,C_BUS,C_NON_MOTOR,C_AIR,T_RAIL,T_CAR,T_COACH,T_BUS,...,AC_RAIL,AC_CAR,AC_COACH,AC_BUS,AC_NON_MOTOR,AC_AIR,MODE,PURPOSE,CAR_AV,DIST
count,86544.000000,86544.000000,86544.000000,86544.000000,86544.0,86544.000000,86544.000000,86544.000000,86544.000000,86544.000000,...,86544.000000,86544.000000,86544.000000,86544.000000,86544.0,86544.000000,86544.000000,86544.000000,86544.000000,86544.000000
mean,15.749310,3.655239,4.183723,2.375040,0.0,980.724610,338.151273,50.362075,668.069721,477.593075,...,0.617631,0.963706,0.617631,0.725333,1.0,0.000595,5.569421,3.634856,0.963914,39.100213
std,28.699297,5.117294,6.351683,0.874815,0.0,128.367185,407.158500,55.813472,415.795417,456.476905,...,0.294744,0.187021,0.294744,0.317794,0.0,0.013502,1.361168,2.054395,0.190913,59.361523
min,0.000000,0.014477,0.001070,0.000000,0.0,43.000000,1.000000,1.000000,10.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,0.010000
25%,5.493301,1.286729,1.474460,2.740000,0.0,1000.000000,61.933333,25.000000,161.350000,71.950000,...,0.500000,1.000000,0.500000,0.750000,1.0,0.000000,6.000000,1.000000,1.000000,13.780000
50%,8.661957,2.344300,2.541250,2.740000,0.0,1000.000000,88.516667,35.000000,1000.000000,119.800000,...,0.700000,1.000000,0.700000,0.900000,1.0,0.000000,6.000000,3.000000,1.000000,23.750000
75%,14.707667,4.119541,4.269300,2.740000,0.0,1000.000000,1000.000000,55.000000,1000.000000,1000.000000,...,0.850000,1.000000,0.850000,0.900000,1.0,0.000000,6.000000,6.000000,1.000000,39.900000
max,1000.000000,106.812632,96.567500,2.740000,0.0,1000.000000,1000.000000,705.033333,1000.000000,1000.000000,...,0.900000,1.000000,0.900000,0.900000,1.0,0.651786,7.000000,6.000000,9.000000,902.500000


In [6]:
# Scale time to hours
df['T_RAIL_S'] = df['T_RAIL'] / 60
df['T_CAR_S'] = df['T_CAR'] / 60
df['T_COACH_S'] = df['T_COACH'] / 60
df['T_BUS_S'] = df['T_BUS'] / 60
df['T_AIR_S'] = df['T_AIR'] / 60
df['T_NON_MOTOR_S'] = df['T_NON_MOTOR'] / 60

In [7]:
# Make car availability binary
df['CAR_AV'] = df['CAR_AV'].replace({9:0})

In [8]:
# Remove trips where mode is car but the car availability is zero
# because it irritates the MLE algorithm
mask = ((df['MODE']==6) & (df['CAR_AV']==0))
print('Share of car trips dropped: {}'.format(
    len(df.loc[mask])/len(df.loc[df['MODE']==6])))
df = df.loc[~mask]

Share of car trips dropped: 0.013851200842412216


In [9]:
'''# Create availability column for air transport (drops 50% of air trips)
# All PT modes are accessible by assumption
df['AIR_AV'] = df['AC_AIR'] > 0
df['AIR_AV'] = df['AIR_AV'].astype(int)
# Remove air trips with 0 availability
df = df.loc[~((df['MODE']==5) & (df['AIR_AV']==0))]'''
len(df.index)

85518

In [10]:
# Remove trips where cost are infinity
# Share of drops per mode must be equal, otherwise the calibration is skewed
# Ignore air trips because this dataset has too few observations anyways
max_drop_ratio = 0
lengths = []
for mode, col in zip(range(1,5), ['C_RAIL', 'C_RAIL', 'C_COACH', 'C_BUS']):
    drops = df.loc[((df['MODE']==mode) & (df[col]==inf))].index
    lengths = lengths + list(df.loc[drops, 'DIST'])
    if len(drops) > 0: print('mode ' + str(mode) + ': ' + str(len(drops)) + ' drops')
    max_drop_ratio = max(len(drops) / len(df.loc[df['MODE']==mode]), max_drop_ratio)
print('max_drop_ratio: ' + str(max_drop_ratio))
print('Dropped trips length mean: {}; min: {}; max: {}'.format(
    sum(lengths)/len(lengths), min(lengths), max(lengths)))

mode 2: 26 drops
max_drop_ratio: 0.02905027932960894
Dropped trips length mean: 35.26923076923077; min: 9.0; max: 86.4


In [11]:
# Drop trips with infinite cost
df = df.loc[~((df['MODE']==2) & (df['C_RAIL']==inf))]
# Don't drop air trips because there are only a few
for m in [1, 3, 4, 6]:
    n_drops = int(max_drop_ratio * len(df.loc[df['MODE']==m]))
    df = df.drop(df.loc[(df['MODE']==m) & (df['DIST']<max(lengths)) &
                        (df['DIST']>min(lengths))].sample(n_drops).index)
    print('mode ' + str(m) + ': ' + str(n_drops) + ' drops')
print('New number of observations: ' + str(len(df.index)))

mode 1: 168 drops
mode 3: 6 drops
mode 4: 92 drops
mode 6: 2122 drops
New number of observations: 83104


### Model variables
All columns are variables. DefineVariable creates a new column in the database.

#### Cost damping

Many modelling studies have shown that cost damping is required in order to flatten the tail of time/cost elasticities, i.e. decrease the impact of long distances on choice results to prevent from overestimation of time/cost parameters. Cost damping represents the property of decreasing marginal utility. It is commonly approached with Box-Cox transformations of generalised cost (usually defined as the sum of travel time and travel expenditures divided by the value of time). Daly (2010) proposes a hybrid function as a sum of the linear term and the narural logarithm of the same. However, the linear term still dominates cost on long distances. Rich (2015), main developer of the Danish National Transport Model, proposes a more complex spline function which successfully manages cost damping and even outperforms the Box-Cox transformation in terms of stability of elasticities.

In [12]:
# Using the hybrid linear-logarithmic version requires a new column
# for the logarithm of time in minutes
'''df['T_RAIL_S'] = np.log(df['T_RAIL'])
df['T_CAR_S'] = np.log(df['T_CAR'])
df['T_COACH_S'] = np.log(df['T_COACH'])
df['T_BUS_S'] = np.log(df['T_BUS'])
df['T_AIR_S'] = np.log(df['T_AIR'])
df['T_NON_MOTOR_S'] = np.log(df['T_NON_MOTOR'])'''
print('This functional form performes poorly')

This functional form performes poorly


#### Generalised cost

Both, travel time and monetary cost should be affected by cost damping measures. It is logical to define a generalised cost term `GC` with dimension of time units. This requires definition or estimation of values of time, in order to rescale monetary units, for all segments. Usually, the value of time (VoT) is distance-dependent. In the case of Germany, VoT can be taken from research undertaken within the Federal Government's transport study "Bundesverkehrswegeplan 2030": Axhausen et al. 2015. Ermittlung von Bewertungsansätzen für Reisezeiten und Zuverlässigkeit auf der Basis eines Modells für modale Verlagerungen im nicht-gewerblichen und gewerblichen Personenverkehr für die Bundesverkehrswegeplanung

In [13]:
# VoT from literature, distance-dependent, see cal19
VoT = pd.read_csv(input_path + 'vot.csv', header=[0,1], index_col=0)
VoT.sample(2)

,root,Fz1,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,...,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,Fz4,Fz6
,all,all,all,all,all,all,PT,PT,PT,PT,...,air,air,air,air,car,car,car,car,car,car
577,18.9749,18.1733,9.18,11.9,23.8844,14.4307,14.3323,11.0782,13.48,15.5082,...,58.7265,58.7265,123.0231,33.6338,18.7941,17.8671,11.3213,23.8743,20.5905,14.4307
590,19.1530,18.3410,9.18,11.9,24.2380,14.5490,14.4610,11.1640,13.48,15.5940,...,60.2150,60.2150,128.7470,34.4060,18.9670,18.0270,11.4110,24.0810,20.8050,14.5490


In [14]:
# Make distance integer
df['DIST'] = df['DIST'].apply(int)

In [15]:
# Generate generalised cost
VoT = VoT.to_dict()
df['GC_RAIL'] = df['T_RAIL_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                  for c,d,p in zip(df['C_RAIL'], df['DIST'], df['PURPOSE'])]
df['GC_COACH'] = df['T_COACH_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                    for c,d,p in zip(df['C_COACH'], df['DIST'], df['PURPOSE'])]
df['GC_BUS'] = df['T_BUS_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                for c,d,p in zip(df['C_BUS'], df['DIST'], df['PURPOSE'])]
df['GC_AIR'] = df['T_AIR_S'] + [c / VoT[('Fz'+str(p), 'air')][d]
                                for c,d,p in zip(df['C_AIR'], df['DIST'], df['PURPOSE'])]
df['GC_CAR'] = df['T_CAR_S'] + [c / VoT[('Fz'+str(p), 'car')][d]
                                for c,d,p in zip(df['C_CAR'], df['DIST'], df['PURPOSE'])]
df['GC_NON_MOTOR'] = df['T_NON_MOTOR_S']

In [16]:
# Create the initial database and make columns global variables
database = db.Database('MiD2017', df.copy())
globals().update(database.variables)
database.getSampleSize()

83104

### Estimation parameters

In [17]:
asc_rail = ex.Beta('asc_rail', 0, None, None, 0)
asc_coach = ex.Beta('asc_coach', 0, None, None, 0)
asc_bus = ex.Beta('asc_bus', 0, None, None, 0)
asc_air = ex.Beta('asc_air', 0, None, None, 0)
asc_car = ex.Beta('asc_car', 0, None, None, 1)
asc_non_motor = ex.Beta('asc_non_motor', 0, None, None, 0)

In [18]:
b_t = ex.Beta('b_t', 0, None, None, 0)
b_c = ex.Beta('b_c', 0, None, None, 0)
b_ac = ex.Beta('b_ac', 0, None, None, 0)

In [19]:
# non-linear time component
b_t2 = ex.Beta('b_t2', 0, None, None, 0)

In [20]:
# generalised cost function parameter
b_gc = ex.Beta('b_gc', 0, None, None, 0)

In [21]:
# Parameters for the nested logit structure
mu_pt = ex.Beta('mu_pt', 1, 1, 10, 0)
mu_long = ex.Beta('mu_long', 1, 1, 10, 0)
mu_short = ex.Beta('mu_short', 1, 1, 10, 0)

### Utility functions

In [23]:
# Aggregated formulation
V_RAIL = asc_rail + b_t * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR_S

In [24]:
# Aggregated formulation with non-linear perception of travel time
V_RAIL = asc_rail + b_t * T_RAIL + b_t2 * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH + b_t2 * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS + b_t2 * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR + b_t2 * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR + b_t2 * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR + b_t2 * T_NON_MOTOR_S

In [22]:
# The cost damping function
def spline(x, beta=b_gc, Q=3, c0=0, c1=20, c2=40, c3=np.inf):
    alpha = [0, -beta/2*ex.Power(ex.log(c1),3),
             -beta/2*ex.log(c1)*(3*ex.Power(ex.log(c2),2)+ex.Power(ex.log(c1),2))] # for Q=3
    theta = [1, 3/2*ex.log(c1), 3*ex.log(c1)*ex.log(c2)] # for Q=3
    def component(x, q):
        return beta*theta[q-1]*ex.Power(ex.log(x),Q-q+1) + alpha[q-1]
    if x < c1:
        return component(x,1)
    elif x < c2:
        return component(x,2)
    elif x >= c2:
        return component(x,3)
    #return sum([component(x, q) if x>=c[q-1] and x<c[q] else 0 for q in range(1,Q+1)])

In [23]:
# Aggregated formulation with damped generalised cost
V_RAIL = asc_rail + spline(GC_RAIL) + b_ac * AC_RAIL
V_COACH = asc_coach + spline(GC_COACH) + b_ac * AC_COACH
V_BUS = asc_bus + spline(GC_BUS) + b_ac * AC_BUS
V_AIR = asc_air + spline(GC_AIR) + b_ac * AC_AIR
V_CAR = asc_car + spline(GC_CAR)
V_NON_MOTOR = asc_non_motor + spline(GC_NON_MOTOR)

### Run the estimation

In [24]:
# Define level of verbosity
logger = message.bioMessage()
#logger.setSilent()
logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

In [25]:
# Map modes to utility functions
V = {1:V_RAIL,
     2:V_RAIL,
     3:V_COACH,
     4:V_BUS,
     5:V_AIR,
     6:V_CAR,
     7:V_NON_MOTOR}

In [26]:
# Map the availability of alternatives with MODE as key
# PT is always available
av = {1:1,
      2:1,
      3:1,
      4:1,
      5:1,
      6:CAR_AV,
      7:1}

In [69]:
# Mode nests as tuples with nest name and dictionary where
# alternative IDs are mapped to alpha values. Missing ID's alpha is zero
# Alternatively use lists with mode ID without alpha
pt = mu_pt, [1, 2, 3, 4]
air = 1, [5]
car = 1, [6]
walk = 1, [7]
nests = pt, air, car, walk

In [70]:
# Choose the multinomial logit model
mnl = models.loglogit(V, av, MODE)

In [71]:
model_mnl = bio.BIOGEME(database, mnl)
model_mnl.modelName = 'MNL'

In [72]:
results_mnl = model_mnl.estimate()

In [73]:
results = results_mnl.getEstimatedParameters()
for key, val in results_mnl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-4.657066,0.412654,-11.2856,0,0.519418,-8.96593,0
asc_bus,-3.755339,0.0789383,-47.5731,0,0.0968411,-38.7784,0
asc_coach,-4.519213,0.105918,-42.6672,0,0.120704,-37.4404,0
asc_non_motor,-1.931669,0.040157,-48.103,0,0.0367954,-52.4976,0
asc_rail,-2.313164,0.0656727,-35.2226,0,0.0769972,-30.0422,0
b_ac,0.579191,0.0878491,6.59302,4.30975e-11,0.108521,5.33714,9.44225e-08
b_gc,-0.313515,0.00784597,-39.9588,0,0.0116377,-26.9397,0
Number of estimated parameters,7.000000,,,,,,
Sample size,31744.000000,,,,,,
Excluded observations,51360.000000,,,,,,


In [74]:
# Write results to a file
writer = pd.ExcelWriter(input_path + 'estimation_results_splineGC_distVoT_knots2040.xlsx', engine='xlsxwriter')

In [75]:
results.to_excel(writer, sheet_name=model_mnl.modelName)

In [76]:
# Choose the logarithmic nested logit model
nl = models.lognested(V, av, nests, MODE)

In [77]:
# Nested Logit
model_nl = bio.BIOGEME(database, nl)
model_nl.modelName = 'NL'

In [78]:
results_nl = model_nl.estimate()

In [79]:
results = results_nl.getEstimatedParameters()
for key, val in results_nl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-4.831625,0,0.420846,-11.4808,0,0.562762,-8.58556,0
asc_bus,-3.882687,0,0.0850133,-45.6715,0,0.100745,-38.5396,0
asc_coach,-4.676038,0,0.134531,-34.7581,0,0.155549,-30.0614,0
asc_non_motor,-1.951645,0,0.0403323,-48.3891,0,0.0366777,-53.2107,0
asc_rail,-3.099025,0,0.0692684,-44.7394,0,0.077857,-39.804,0
b_ac,0.646812,0,0.0852977,7.583,3.37508e-14,0.103271,6.26326,3.77008e-10
b_gc,-0.297957,0,0.010057,-29.6268,0,0.0132947,-22.4118,0
mu_pt,1.000000,1,0.0378214,26.4401,0,0.0434699,23.0044,0
Number of estimated parameters,8.000000,,,,,,,
Sample size,31744.000000,,,,,,,


In [80]:
results.to_excel(writer, sheet_name=model_nl.modelName)

In [81]:
# Run the estimation by purpose
results = []
for p in [1,2,3,4,6]:
    database = db.Database('MiD2017', df.copy())
    database.remove(PURPOSE!=p)
    print('Sample size for purpose {}: {}'.format(p, database.getSampleSize()))
    model = bio.BIOGEME(database, nl) # Choose the model formulation
    model.modelName = 'NL_Fz' + str(p) # Name it
    results.append(model.estimate()) # Estimation
    output = results[-1].getEstimatedParameters()
    # Add results to the Excel file
    for key, val in results[-1].getGeneralStatistics().items():
        output.loc[key] = [val[0], val[1]] + ['' for i in range(len(output.columns)-2)]
    output.to_excel(writer, sheet_name=model.modelName)

Sample size for purpose 1: 22195
Sample size for purpose 2: 3589
Sample size for purpose 3: 20771
Sample size for purpose 4: 4805
Sample size for purpose 6: 31744


In [82]:
writer.save()

In [41]:
# Generate LaTeX code
file = open(input_path + 'estimation_results_LaTeX_code.txt', 'w')
for r in results:
    file.write(r.getLaTeX())
file.close()